In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47228
2,Huelva,Confirmados PDIA 14 días,734
3,Huelva,Tasa PDIA 14 días,"143,03252333534695"
4,Huelva,Confirmados PDIA 7 días,378
5,Huelva,Tasa PDIA 7 dias,"73,65980084572364"
6,Huelva,Total Confirmados,47432
7,Huelva,Curados,45685
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47228.0


/tmp/ipykernel_131362/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13073.0


/tmp/ipykernel_131362/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_131362/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_131362/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_131362/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 985 personas en los últimos 7 días 

Un positivo PCR cada 501 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47228.0,378.0,734.0,513170.0,73.659801,143.032523,109.0
Huelva-Costa,27929.0,226.0,441.0,289548.0,78.052689,152.306353,66.0
Huelva (capital),13073.0,146.0,287.0,143837.0,101.503786,199.531414,36.0
Condado-Campiña,14716.0,128.0,255.0,156231.0,81.929963,163.219848,33.0
Moguer,2104.0,34.0,59.0,21867.0,155.485435,269.812960,15.0
Cartaya,2235.0,35.0,73.0,20083.0,174.276751,363.491510,11.0
Palos de la Frontera,1162.0,30.0,62.0,11742.0,255.493102,528.019077,11.0
Sierra de Huelva-Andévalo Central,4155.0,24.0,37.0,67391.0,35.613064,54.903474,10.0
Aljaraque,1678.0,21.0,29.0,21474.0,97.792680,135.047034,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),13.0,7.0,8.0,243.0,2880.658436,3292.181070,3.0
Palos de la Frontera,1162.0,30.0,62.0,11742.0,255.493102,528.019077,11.0
Paterna del Campo,230.0,14.0,17.0,3457.0,404.975412,491.755858,4.0
Alosno,247.0,12.0,15.0,3933.0,305.110603,381.388253,7.0
Cartaya,2235.0,35.0,73.0,20083.0,174.276751,363.491510,11.0
Campofrío,61.0,1.0,2.0,713.0,140.252454,280.504909,0.0
Moguer,2104.0,34.0,59.0,21867.0,155.485435,269.812960,15.0
San Bartolomé de la Torre,398.0,3.0,10.0,3761.0,79.766020,265.886732,0.0
Palma del Condado (La),1467.0,13.0,27.0,10801.0,120.359226,249.976854,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Isla Cristina,3195.0,2.0,17.0,21393.0,9.348852,79.465246,1.0,0.117647
San Bartolomé de la Torre,398.0,3.0,10.0,3761.0,79.766020,265.886732,0.0,0.300000
Bollullos Par del Condado,1341.0,8.0,24.0,14387.0,55.605755,166.817266,0.0,0.333333
Gibraleón,967.0,5.0,15.0,12737.0,39.255712,117.767135,0.0,0.333333
Ayamonte,2118.0,2.0,5.0,21104.0,9.476876,23.692191,1.0,0.400000
Trigueros,551.0,2.0,5.0,7862.0,25.438820,63.597049,0.0,0.400000
Bonares,473.0,2.0,5.0,6060.0,33.003300,82.508251,0.0,0.400000
Almonte,2366.0,9.0,20.0,24507.0,36.724201,81.609336,0.0,0.450000
Cartaya,2235.0,35.0,73.0,20083.0,174.276751,363.491510,11.0,0.479452
